In [2]:
import os
import json
import openai
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import AzureOpenAI
from llama_index import LangchainEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import GPTVectorStoreIndex, LLMPredictor, PromptHelper
import logging
import sys

In [3]:
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")


In [4]:
print(os.getenv("OPENAI_API_TYPE"))

azure


In [5]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
llm = AzureOpenAI(engine="gpt-35-turbo", model="gpt-3.5-turbo")

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="text-embedding-ada-002",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

documents = SimpleDirectoryReader("data").load_data()

In [7]:
## Selecting subset of the text for demo purposes
((documents[0]).text) = ((documents[0]).text)[0:1000]

In [8]:
((documents[0]).text) 

'\t\t\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, th

In [9]:
llm_predictor = LLMPredictor(llm=llm)

In [10]:
from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

set_global_service_context(service_context)

In [11]:
index = VectorStoreIndex.from_documents(documents)

In [12]:
query = "Which computer did the author use to write the first programs"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

In [13]:
print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

> Source (Doc id: c58772ce-e6e0-4692-9682-62bb9164b9f7): What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...
query was: Which computer did the author use to write the first programs
answer was: The author used the IBM 1401 computer to write the first programs.
